# get n671_u16 signal

In [ ]:
import os,glob
import numpy as np
import pandas as pd

def get_labels(label_brain_file):
    label_brain_csv = pd.read_csv(label_brain_file,)
    label_brain_csv = label_brain_csv.fillna('0')
    label_brain_csv['label'] = label_brain_csv['label'].apply(str)
    label_brain_csv['target'] = label_brain_csv['label'].apply(lambda label:','.join([t.split('-',1)[0] for t in label.split(';',1)[0].split('+')]) if type(label)==type(str(label)) else '')
    label_brain_csv.to_csv(label_brain_file,columns=label_brain_csv.columns,index=False)
    
    brains = label_brain_csv['brain'].apply(str)
    labels = label_brain_csv['target']
    brain_label_dict = dict(zip(brains,labels))
    
    labelnum = label_brain_csv['target'].value_counts().apply(int)
    label_num_dict = dict(zip(labelnum.index,labelnum.values))
    label_num_dict.pop('','no label is empty')
    
    brain_label_dict = {k:v for k,v in brain_label_dict.items() if v==v}
    return brain_label_dict, label_num_dict


def get_brains_signal_df(brains,
                          signal_dir,
                          allow0,
                          sizes_brains_dict):
    size_standard = 528*25*320*25*456*25
    df_brains = pd.DataFrame([])    
    for brain in brains:
        size_scale = size_standard/np.prod(sizes_brains_dict[brain])
        brain_path = f'{signal_dir}/*/{brain}.csv'
        brain_path = glob.glob(brain_path)[0]
        if os.path.exists(brain_path):
            print(f'{brain_path} appended')
            df_brain = pd.read_csv(brain_path,header=0,usecols=[0,1],names=['n671_u16_id','count']) 
            df_brain['n671_u16_id'] = df_brain['n671_u16_id'].apply(int)
            df_brain['sizecount'] = df_brain['count'].apply(int)
            if len(df_brain):
                df_brain['brain'] = str(brain)
                df_brain['scale'] = size_scale
                df_brain['count'] = (df_brain['sizecount']*size_scale).apply(int)
                df_brain['counts'] = int(df_brain['count'].sum())                
                if not allow0:
                    df_brain = df_brain[df_brain['n671_u16_id']!=0] 
                df_brains = pd.concat([df_brains,df_brain])
        else:
            print(f'{brain_path} does not exists')  
    return df_brains   


def get_brains_signal(labels_brains_dict,
                      sizes_brains_dict,
                               signal_dir,
                               allow0,
                               ):
    df_labels = pd.DataFrame([])    
    for label,brains in labels_brains_dict.items():
        df_label = get_brains_signal_df(brains,
                                         signal_dir,
                                         allow0,
                                         sizes_brains_dict)
        if len(df_label):
            df_label['modality'] = label
            df_labels = pd.concat([df_labels,df_label])
    return df_labels  


def get_modality_brain_dict(modality_brain_file):
    modalities_brains_dict = {}
    modality_brain_df = pd.read_csv(modality_brain_file)
    modality_brain_df['source'] = modality_brain_df['source'].apply(str)
    modality_brain_df['ID'] = modality_brain_df['ID'].apply(str)
    modalities = modality_brain_df['source']
    for modality in modalities:
        brains = modality_brain_df['ID'][modality_brain_df['source']==modality].tolist()
        if len(brains):
            modalities_brains_dict[modality] = brains
    return modalities_brains_dict
    
def get_size_brain_dict(size_brain_file):
    sizes_brains_dict = {}
    size_brain_df = pd.read_csv(size_brain_file)
    brains = size_brain_df['ID'].apply(str).tolist()
    xyz_orig = size_brain_df.values[:,[2,1,3]]
    scale = np.abs(size_brain_df.values[:,-1])-1
    xyz_calc = np.divide(xyz_orig,np.repeat([np.power(2,scale)],3,axis=0).T).astype(float)
    xyz_calc = np.round(xyz_calc).astype(int).tolist()
    sizes_brains_dict = dict(zip(brains,xyz_calc))
    return sizes_brains_dict



info_brain_file = '../../assets/TeraDownsampleSize.csv'
modalities_brains_dict = get_modality_brain_dict(info_brain_file)
sizes_brains_dict = get_size_brain_dict(info_brain_file)
signal_dir = '../statis_out_corrected0315'
allow0 = False
df_signal = get_brains_signal(labels_brains_dict = modalities_brains_dict,
                              sizes_brains_dict = sizes_brains_dict,
                              signal_dir = signal_dir,
                              allow0 = allow0,)  


label_brain_file = '../../assets/fMOST-Zeng_labels_edited.csv'
brain_label_dict,label_num_dict = get_labels(label_brain_file)  
df_signal['label'] = df_signal['brain'].apply(lambda b:brain_label_dict[b] if b in brain_label_dict.keys() else '0')



df_signal.to_csv('../../assets/brain205_signal_region671.csv',index=False)
df_signal

# relplot whole-brain signal

In [21]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from utils import get_dict

def plot_brainnum(df_signal):
    data = []
    for label,df_label in df_signal.groupby(by='label'):
        if label=='0': continue
        dfs_brain = df_label.groupby(by='brain')
        label_braincount = len(dfs_brain)    
        data.append([label,label_braincount])
    data = sorted(data,key=lambda x:x[1])
    df = pd.DataFrame(np.array(data),columns=['label','num'])
    df['label'] = df['label'].apply(str)
    df['num'] = df['num'].apply(int)

    height = 3
    width = 8
    plt.figure(figsize=(height,width))
    ax = sns.barplot(data=df,x='num',y='label',orient='h',color='#e377e3')
    ax.set(xticks=[],yticks=[],ylabel='')
    for i,n in zip(list(range(len(df))),df['num'].tolist()):
        ax.text(n,i,n,va='center',fontsize=18)
    sns.despine(left=True,right=True,top=True,bottom=True,)
    plt.xlabel('#brains',fontsize=16)
    plt.tight_layout()    
    plt.savefig('../plots/label_nhist.png', dpi=300)
    plt.close('all') 
    return [d[0] for d in data]
    
def plot_signallevel(df_signal,lines):
    brains = []
    datas = []
    dfs_label = df_signal.groupby(by='label')
    for label in lines:
        data = []
        df_label = [df for l,df in dfs_label if l==label][0]
        dfs_brain = df_label.groupby(by='brain')
        for brain,df_brain in dfs_brain:
            data.append([brain,label,df_brain['count'].sum()])
        data = sorted(data,key=lambda x:x[2])
        brains.append([label,[d[0] for d in data]])
        datas.extend(data)
    df = pd.DataFrame(np.array(datas),columns=['brain','label','counts'])
    df['brain'] = df['brain'].apply(str)
    df['label'] = df['label'].apply(str)
    df['counts'] = df['counts'].apply(int)

    height = 30
    width = 5
    plt.figure(figsize=(width,height))
    sns.set_style("whitegrid")
    sns.scatterplot(data=df, x='counts', y='label', c='m')
    plt.xscale('log')
    plt.xticks(fontsize=18)
    plt.xlabel('#neurite voxels', fontsize=18)
    plt.yticks(fontsize=18)
    plt.ylabel('lines', fontsize=18)
    plt.tight_layout()    
    plt.savefig('../plots/label_signal.png', dpi=300)
    plt.close('all')  
    return brains

def plot_regional_density(df_signal,brains):
    annofile = f'../../assets/n671_u16.nrrd'
    an,_ = get_section_array(annofile,None,180) 
    
    level = 671
    uint = 16
    n671_u16_voxel_dict = get_dict(level,uint,key='voxel')
    voxel_list = [(n671_u16_voxel_dict[i] if i in n671_u16_voxel_dict.keys() else float('inf')) for i in df_signal['n671_u16_id']]
    df_signal['density'] = df_signal['count']/voxel_list#1 um pixel on 25 um volume, should (voxel*25) 
    df_signal['density'] = df_signal['density']/(0.025*0.025*0.025)
    df_signal['logdensity'] = np.log10(df_signal['density']+1).round(2)
    md = df_signal['logdensity'].max()
    nc = 20
    df_signal['ncolor'] = (nc*df_signal['logdensity']/md).astype(int)
    df_signal['colorcode'] = get_colorcode16_list(df_signal['ncolor'],nc=nc,cmap='spring',reverse=True)
    for i in range(nc+1):
        j = df_signal['logdensity'][df_signal['ncolor']==i].max()
        k = df_signal['density'][df_signal['ncolor']==i].max()
        print(i,j,k)    
    for l,bs in brains:
        ib = 0
        fig = plt.figure(figsize=(3*len(bs),3))        
        for b in bs:
            anno = an.copy()
            df_brain = df_signal[df_signal['brain']==b]     
            if os.path.exists(annofile):
                ib += 1
                fig.add_subplot(1,len(bs),ib)
                for region,color in zip(df_brain['n671_u16_id'],df_brain['colorcode']):
                    mask = anno==region
                    indices = np.nonzero(mask)
                    plt.scatter(indices[0], indices[1], s=0.5, c='#'+color, alpha=1.0, edgecolors='none')
                print(f'{b} color mapping done')
                boundary = get_section_boundary(anno)
                indices = np.nonzero(boundary)
                plt.scatter(indices[0], indices[1], s=0.1, c='black', alpha=1.0, edgecolors='none')
                print(f'{b} boundary plotting done')
                plt.axis('off')
        plt.savefig(f'../plots/{l}_density.png',dpi=300) 
        plt.close()   
    return df_signal   

def plot_brain_consistency(df_signal,brains):#brain consistency of regional density corr
    level = 671
    uint = 16
    n671_u16_voxel_dict = get_dict(level,uint,key='voxel')
    voxel_list = [(n671_u16_voxel_dict[i] if i in n671_u16_voxel_dict.keys() else float('inf')) for i in df_signal['n671_u16_id']]
    df_signal['density'] = df_signal['count']/voxel_list#1 um pixel on 25 um volume, should (voxel*25) 
    df_signal['density'] = df_signal['density']/(0.025*0.025*0.025)
    df_signal_new = pd.pivot_table(df_signal,index=['n671_u16_id'],columns=['brain'],values='density',aggfunc='sum',fill_value=0)
    df_signal_new.columns = df_signal_new.columns.astype(str)
    df_signal_new.index = df_signal_new.index.astype(int)
    df_signal_new = df_signal_new.iloc[df_signal_new.index!=0] 
    
    colorcode = get_colorcode16_list(list(range(11)),nc=10,cmap='spring',reverse=True)
    nl = len([l for l,bs in brains if len(bs)>1])
    data = []
    il = 0
    fig = plt.figure(figsize=(10,nl*2))    
    for label,bs in brains:
        dfs_brain = df_signal_new[bs] 
        scores = calc_score(dfs_brain)
        if len(scores)>0:
            il += 1
            mean = sum(scores)/len(scores)
            data.append(mean)  
            ax = fig.add_subplot(nl,1,il)
            sns.boxplot(scores,fliersize=1,orient='h',color='#'+colorcode[int(round(min(scores)*10))],ax=ax)            
            print(il,label)
            ax.spines['top'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.set_ylim([-1,2])
            ax.set_yticks([])
            ax.set_ylabel('')#'   '+str(round(mean,2)),fontsize=40,rotation=0)
            #ax.text(round(mean,2),il,round(mean,2),fontsize=40,rotation=0)
            ax.set_xlim([0,1])
            ax.set_xticklabels(['0','','','','1'],fontsize=32)
            ax.set_xlabel('')    

    plt.subplots_adjust(left=0.5,hspace=2)
    plt.savefig(f'../plots/intra-line-brain-consistency-on-regional-distri-corr.png',dpi=300)  
    plt.close()
    return data
    
def plot_wholesignal(df_signal):
    lines = plot_brainnum(df_signal)
    print(lines)
    brains = plot_signallevel(df_signal,lines)
    print(brains)
    #plot_regional_density(df_signal,brains)
    tmp = plot_brain_consistency(df_signal,brains)
    return tmp
    
df_signal = pd.read_csv('../../assets/brain205_signal_region671.csv')
df_signal['n671_u16_id'] = df_signal['n671_u16_id'].astype(int)
df_signal['count'] = df_signal['count'].astype(int)
df_signal['brain'] = df_signal['brain'].astype(str)
df_signal['modality'] = df_signal['modality'].astype(str)
df_signal['label'] = df_signal['label'].astype(str)

plot_wholesignal(df_signal)

# fig = sns.palplot(sns.color_palette('spring',101),)
# plt.axis('off')
# plt.savefig(f'../plots/spring_cbar.png',dpi=300)
# plt.close()

/home/lyf/anaconda3/envs/pytorch1.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['Chrna2', 'Ctgf', 'Ctxn3', 'Esr2', 'Gal', 'Gpr139', 'Grp', 'Syn', 'Vip', 'iCreV', 'Chrna6', 'Etv1,Pvalb', 'Nos1,Sst', 'Oxtr,Pvalb', 'Chrna2,Sst', 'Pdyn,Sst', 'Tacr1,Sst', 'Tlx3', 'Dbh', 'Slc6a4', 'Th', 'Tle4', 'Gnb4', 'Npr3', 'Sst', 'Nxph4', 'Tnnt1', 'Vipr2', 'Chat', 'Pdyn', 'Pvalb', 'Cux2', 'Fezf2', 'Plxnd1', 'Calb2']
[['Chrna2', ['17783']], ['Ctgf', ['211791']], ['Ctxn3', ['211549']], ['Esr2', ['18467']], ['Gal', ['211544']], ['Gpr139', ['211541']], ['Grp', ['17785']], ['Syn', ['182712']], ['Vip', ['211540']], ['iCreV', ['182711']], ['Chrna6', ['211796', '211550']], ['Etv1,Pvalb', ['18472', '18471']], ['Nos1,Sst', ['182727', '182726']], ['Oxtr,Pvalb', ['18866', '18865']], ['Chrna2,Sst', ['191798', '191799', '191797']], ['Pdyn,Sst', ['191810', '191809', '191811']], ['Tacr1,Sst', ['192348', '192346', '192349']], ['Tlx3', ['17304', '17543', '17544']], ['Dbh', ['191804', '182721', '182720', '182722']], ['Slc6a4', ['18049', '18466', '18459', '17301']], ['Th', ['18871', '211798', '211546'

<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)


6 Pdyn,Sst
7 Tacr1,Sst
8 Tlx3
9 Dbh
10 Slc6a4
11 Th
12 Tle4
13 Gnb4
14 Npr3


<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-i

15 Sst
16 Nxph4
17 Tnnt1
18 Vipr2
19 Chat
20 Pdyn
21 Pvalb


<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-i

22 Cux2
23 Fezf2
24 Plxnd1
25 Calb2


<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)
<ipython-input-21-a07ba749c94b>:145: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(['0','','','','1'],fontsize=32)


[0.6398859353738298,
 0.9118747522041274,
 0.4525799626104083,
 0.6147518337687301,
 0.7580884078409097,
 0.6667864484334256,
 0.7376916881658405,
 0.12669834523906284,
 0.2557063070879532,
 0.23981008869003884,
 0.24064821875182577,
 0.32329044334428997,
 0.3161838118193403,
 0.3892142038990935,
 0.24053523448836586,
 0.5194610748837771,
 0.4350796626384369,
 0.6012089920423739,
 0.26760269238338696,
 0.4211695423039205,
 0.6215738781286866,
 0.1966933003467577,
 0.39418198494055745,
 0.19302662576767451,
 0.5174192826629979]

In [8]:
import numpy as np
from file_io import load_image
import seaborn as sns
import imageio

def rotate3d(img3d):
    img = np.zeros((img3d.shape[1],img3d.shape[0],img3d.shape[2]))
    for i in range(img3d.shape[2]):
        img[:,:,i] = img3d[:,:,i].T
    img = img.astype(np.uint8)
    return img

def rotate2d(img2d,rot=90):
    k = int(rot/90)
    img = np.rot90(img2d,k)
    return img

def get_section_array(imagefile,axis=None,rot=0):
    if imagefile.endswith('nrrd'): image = load_image(imagefile)
    else: image = load_image(imagefile)[0]
    if axis==None:
        axis = image.shape.index(min(image.shape))
        if axis==0:
            section = eval(f'image[{int(image.shape[axis]/2)},:,:]')
        elif axis==1:
            section = eval(f'image[:,{int(image.shape[axis]/2)},:]')
        else:
            section = eval(f'image[:,:,{int(image.shape[axis]/2)}]')
    imageio.imsave('test.png',section)
    if rot:
        return rotate2d(section,rot),axis
    else:
        return section,axis
    
def detect_edges2d(img2d):
    img = img2d.astype(float)
    gx, gy = np.gradient(img)
    edges = (gx*gx + gy*gy) != 0
    return edges

def get_brain_outline2d(mask, axis=0, v=255):
    mask = mask > 0
    mask2d = mask.max(axis=axis)
    outline = detect_edges2d(mask2d)
    return outline.astype(np.uint8) * v

def get_section_boundary(array,axis=1,v=255,withoutline=False):
    if len(array.shape)==3:
        if axis==0:
            section = eval(f'array[{int(array.shape[axis]/2)},:,:]')
        elif axis==1:
            section = eval(f'array[:,{int(array.shape[axis]/2)},:]')
        else:
            section = eval(f'array[:,:,{int(array.shape[axis]/2)}]')
        boundary = detect_edges2d(section)
        if withoutline:
            outline = get_brain_outline2d(array, axis=axis, v=v)    
            return np.maximum(boundary.astype(np.uint8) * v  , outline)
    else:
        boundary = detect_edges2d(array)
    return boundary.astype(np.uint8) * v  

def get_colorcode16_list(value_list,nc=10,cmap='spring',reverse=True):        
    rgblist = (np.array(sns.color_palette(cmap,nc+1))*255).astype(int)
    if reverse: 
        rgblist = rgblist[nc::-1,:]
    code16_list = ["".join([hex(c)[2:].rjust(2,'0').upper() for c in rgb]) for rgb in rgblist]
    value_code16_list = [code16_list[v] for v in value_list]
    return value_code16_list

def calc_score(df):
    df_new = df.replace(0, np.nan)
    corr = df_new.corr(min_periods=20, method='spearman')
    corr = corr.fillna(0)
    corr = abs(corr)    
    triu = np.triu(np.ones(corr.shape),1)
    scores = corr.values[triu>0].tolist()
    return scores